In [1]:
# Do all imports and installs here - Done
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
import pandas as pd
import re
import configparser
import os

Parse config file for path configurations - Done

In [2]:
config = configparser.ConfigParser()
config.read('etl.cfg')

input_data_source = config.get('DIR','INPUT_DIR')
output_processed_data = config.get('DIR','OUTPUT_DIR')

i94immi_dataset = config.get('DATA','I94_IMMI')
worldtempe_dataset = config.get('DATA','WORLD_TEMPE')
citydemo_dataset = config.get('DATA','CITY_DEMOGRAPHIC')
airport_dataset = config.get('DATA','AIR_PORT')

- Extract dictionary informations from *I94_SAS_Labels_Descriptions.SAS*.
    - I94CIT & I94RES --> i94cntyl.txt
    - I94PORT --> i94prtl.txt
    - I94MODE --> i94model.txt
    - I94ADDR --> i94addrl
    - I94VISA --> i94visa.txt

In [3]:
with open('./I94_SAS_Labels_Descriptions.SAS') as f:
    i94_sas_labels = f.read()
    i94_sas_labels = i94_sas_labels.replace('\t', '')

In [4]:
def code_mapper(file, idx):
    i94_sas_labels_content = i94_sas_labels[i94_sas_labels.index(idx):]
    i94_sas_labels_content = i94_sas_labels_content[:i94_sas_labels_content.index(';')].split('\n')
    i94_sas_labels_content = [i.replace("'", "") for i in i94_sas_labels_content]
    dic = [i.split('=') for i in i94_sas_labels_content[1:]]
    dic = dict([i[0].strip(), i[1].strip()] for i in dic if len(i) == 2)
    return dic

In [5]:
i94_cit_and_res = code_mapper(i94_sas_labels, "i94cntyl")
i94_port = code_mapper(i94_sas_labels, "i94prtl")
i94_mode = code_mapper(i94_sas_labels, "i94model")
i94_addr = code_mapper(i94_sas_labels, "i94addrl")
i94_visa = {'1':'Business',
            '2': 'Pleasure',
            '3' : 'Student'}

In [20]:
i94_cit_and_res

{'582': 'MEXICO Air Sea, and Not Reported (I-94, no land arrivals)',
 '236': 'AFGHANISTAN',
 '101': 'ALBANIA',
 '316': 'ALGERIA',
 '102': 'ANDORRA',
 '324': 'ANGOLA',
 '529': 'ANGUILLA',
 '518': 'ANTIGUA-BARBUDA',
 '687': 'ARGENTINA',
 '151': 'ARMENIA',
 '532': 'ARUBA',
 '438': 'AUSTRALIA',
 '103': 'AUSTRIA',
 '152': 'AZERBAIJAN',
 '512': 'BAHAMAS',
 '298': 'BAHRAIN',
 '274': 'BANGLADESH',
 '513': 'BARBADOS',
 '104': 'BELGIUM',
 '581': 'BELIZE',
 '386': 'BENIN',
 '509': 'BERMUDA',
 '153': 'BELARUS',
 '242': 'BHUTAN',
 '688': 'BOLIVIA',
 '717': 'BONAIRE, ST EUSTATIUS, SABA',
 '164': 'BOSNIA-HERZEGOVINA',
 '336': 'BOTSWANA',
 '689': 'BRAZIL',
 '525': 'BRITISH VIRGIN ISLANDS',
 '217': 'BRUNEI',
 '105': 'BULGARIA',
 '393': 'BURKINA FASO',
 '243': 'BURMA',
 '375': 'BURUNDI',
 '310': 'CAMEROON',
 '326': 'CAPE VERDE',
 '526': 'CAYMAN ISLANDS',
 '383': 'CENTRAL AFRICAN REPUBLIC',
 '384': 'CHAD',
 '690': 'CHILE',
 '245': 'CHINA, PRC',
 '721': 'CURACAO',
 '270': 'CHRISTMAS ISLAND',
 '271': 'COCO

In [8]:
i94_port

{'ALC': 'ALCAN, AK',
 'ANC': 'ANCHORAGE, AK',
 'BAR': 'BAKER AAF - BAKER ISLAND, AK',
 'DAC': 'DALTONS CACHE, AK',
 'PIZ': 'DEW STATION PT LAY DEW, AK',
 'DTH': 'DUTCH HARBOR, AK',
 'EGL': 'EAGLE, AK',
 'FRB': 'FAIRBANKS, AK',
 'HOM': 'HOMER, AK',
 'HYD': 'HYDER, AK',
 'JUN': 'JUNEAU, AK',
 '5KE': 'KETCHIKAN, AK',
 'KET': 'KETCHIKAN, AK',
 'MOS': 'MOSES POINT INTERMEDIATE, AK',
 'NIK': 'NIKISKI, AK',
 'NOM': 'NOM, AK',
 'PKC': 'POKER CREEK, AK',
 'ORI': 'PORT LIONS SPB, AK',
 'SKA': 'SKAGWAY, AK',
 'SNP': 'ST. PAUL ISLAND, AK',
 'TKI': 'TOKEEN, AK',
 'WRA': 'WRANGELL, AK',
 'HSV': 'MADISON COUNTY - HUNTSVILLE, AL',
 'MOB': 'MOBILE, AL',
 'LIA': 'LITTLE ROCK, AR (BPS)',
 'ROG': 'ROGERS ARPT, AR',
 'DOU': 'DOUGLAS, AZ',
 'LUK': 'LUKEVILLE, AZ',
 'MAP': 'MARIPOSA AZ',
 'NAC': 'NACO, AZ',
 'NOG': 'NOGALES, AZ',
 'PHO': 'PHOENIX, AZ',
 'POR': 'PORTAL, AZ',
 'SLU': 'SAN LUIS, AZ',
 'SAS': 'SASABE, AZ',
 'TUC': 'TUCSON, AZ',
 'YUI': 'YUMA, AZ',
 'AND': 'ANDRADE, CA',
 'BUR': 'BURBANK, CA',
 '

In [21]:
i94_mode

{'1': 'Air', '2': 'Sea', '3': 'Land', '9': 'Not reported'}

In [ ]:
i94_addr

In [6]:
i94_visa

{'1': 'Business', '2': 'Pleasure', '3': 'Student'}

In [ ]:
# Create spark session
spark = SparkSession.builder\
            .config("spark.jars.repositories", "https://repos.spark-packages.org/")\
            .config("spark.jars.packages", "saurfang:spark-sas7bdat:2.0.0-s_2.11")\
            .enableHiveSupport()\
            .getOrCreate()

In [13]:
# Reading the clean temperature data csv with spark session - must be spark
worldtempe_dataset = 'worldtempe_df_clean.csv'
worldtempe_df = pd.read_csv(worldtempe_dataset,sep=",")

In [14]:
worldtempe_df.head()

dt  averagetemperature      city        country  year  month
0  1828-01-01              -1.977  Bellevue  United States  1828      1
1  1828-02-01               0.720  Bellevue  United States  1828      2
2  1828-03-01               4.114  Bellevue  United States  1828      3
3  1828-05-01              10.139  Bellevue  United States  1828      5
4  1828-06-01              14.195  Bellevue  United States  1828      6

In [ ]:
# Testing
def convert_city_to_i94port(city):

    for port in i94_port:
        if city.lower() in i94_port[port].lower():
            return port


worldtempe_df = worldtempe_df.withColumn('i94_port', convert_city_to_i94port(worldtempe_df['city']))

In [18]:
# Testing
def convert_city_to_i94port(city):
    results = [v for k, v in i94_port.items() if re.match(city, k)]

In [ ]:
from pyspark.sql.functions import udf,col
convert_city_to_i94portUDF = udf(lambda x:convert_city_to_i94port(x))

worldtempe_with_i94port_df = worldtempe_df.withColumn('i94_port', convert_city_to_i94portUDF(col("city")))
worldtempe_with_i94port_df.show(2)

In [ ]:
# Create udf to map city full name to city port abbreviation

@udf(StringType())
def city_to_port(city):
    for key in valid_ports:
        if city.lower() in valid_ports[key].lower():
            return key

In [ ]:
@udf()
def get_i94port(city):
    '''
    Input: City name 
    Output: Corresponding i94port
    '''
    
    for key in i94portvalid:
        if city.lower() in i94portvalid[key][0].lower():
            return key

In [ ]:
# # Mapping the U.S cities to city port abbreviations (i94port from SAS label)

# # Add iport94 code based on city name
df_temperature_data = df_temperature_data.withColumn("i94port", get_i94port(df_temperature_data.City))

# Remove data points with no iport94 code
df_temperature_data = df_temperature_data.filter(df_temperature_data.i94port != 'null')

In [ ]:
# Clean temperature data

# Only use temperatures from United States
# Map full name to city port abbreviation
# Remove invalid ports
cleaned_temp_df = temperature_df.filter(temperature_df["Country"] == "United States") \
    .withColumn("year", year(temperature_df['dt'])) \
    .withColumn("month", month(temperature_df["dt"])) \
    .withColumn("i94port", city_to_port(temperature_df["City"])) \
    .withColumn("AverageTemperature", col("AverageTemperature").cast("float")) \
    .dropna(how='any', subset=["i94port"])

In [ ]:
# Add iport94 code based on city name
df_temperature_data = df_temperature_data.withColumn("i94port", get_i94port(df_temperature_data.City))
df_temperature_data.show()